# Notebook to generate psychometrics plot given sessions & conditions

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import pandas as pd
import time
from zoneinfo import ZoneInfo
import datetime
import pathlib
import u19_pipeline.utils.slack_utils as su


c:\Users\MiniVR-8\miniconda3\envs\u19_pipeline_python_env\Lib\site-packages\datajoint\plugin.py:4: UserWarning: Module scripts was already imported from None, but c:\experiments\u19-pipeline-python is being added to sys.path
  import pkg_resources


In [13]:
def get_subject_data():

    #Query from file
    current_directory = pathlib.Path.cwd()
    query_file = pathlib.Path(current_directory, 'useful_queries', 'get_subject_data.sql').as_posix()

    with open(query_file, "r", encoding="utf-8") as file:

        subject_query = file.read()

    conn = dj.conn()
    subject_data = pd.DataFrame(conn.query(subject_query, as_dict=True).fetchall())



    #Get subject fullname only once
    subject_data = subject_data.loc[:,~subject_data.columns.duplicated()]
    #subject_data = subject_data.rename({'new_subject_fullname': 'subject_fullname'}, axis=1)

    # Get column list
    cols = subject_data.columns.tolist()

    # Remove and insert the column
    cols.insert(0, cols.pop(cols.index('subject_fullname')))
    subject_data = subject_data.reindex(columns=cols)


    #Get today's responsibility based on schedule "Nothing/Nothing etc" field
    today_idx  = (datetime.datetime.today().weekday() + 1) % 7
    subject_data['schedule_today'] = subject_data['schedule'].str.split('/')
    subject_data['schedule_today'] = subject_data['schedule_today'].apply(lambda x: x[today_idx])



    subject_data = subject_data.loc[(((subject_data['schedule_today'] != "Nothing") &\
                                        (subject_data['subject_status'] == "InExperiments")) |\
                                    (subject_data['subject_status'] == "WaterRestrictionOnly")), :]

    subject_data = subject_data.reset_index(drop=True)



    with pd.option_context("future.no_silent_downcasting", True):

        subject_data['earned'] = subject_data['earned'].fillna(0).infer_objects(copy=False)
        subject_data['received'] = subject_data['received'].fillna(0).infer_objects(copy=False)
        subject_data['supplement'] = subject_data['supplement'].fillna(0).infer_objects(copy=False)
        subject_data['prescribed_extra_supplement_amount'] = subject_data['prescribed_extra_supplement_amount'].fillna(0).infer_objects(copy=False)


    #Calculated fields 
    subject_data['already_water'] = subject_data['supplement']>0
    subject_data['upper_cage'] = subject_data['cage'].str.upper()
    subject_data['total_water_received'] = subject_data['received']
    subject_data.loc[subject_data['total_water_received'].isnull(), 'total_water_received'] = 0

    subject_data['need_supplement'] = 0
    subject_data['need_supplement'] = (subject_data['total_water_received'] < (subject_data['water_per_day']  -0.05)) & ~subject_data['already_water']

    subject_data.loc[subject_data['already_received'].isnull(), 'already_received'] = 0
    subject_data['need_extra_water_now'] = 0
    subject_data.loc[((subject_data['prescribed_extra_supplement_amount'] > 0) & (subject_data['already_received'] == 0)),'need_extra_water_now'] = 1

    subject_data['water_status'] = "Already Watered"
    subject_data.loc[subject_data['need_supplement'] == 1, 'water_status'] = "Need Supplement"
    subject_data.loc[subject_data['need_extra_water_now'] == 1, 'water_status'] = "Need Extra Supplement"

    subject_data['need_water'] = 0
    subject_data.loc[(subject_data['need_supplement'] | subject_data['need_extra_water_now']), 'need_water'] = 1

    subject_data['current_need_water'] = subject_data['suggested_water']
    subject_data.loc[subject_data['need_extra_water_now']==1, 'current_need_water'] =\
    subject_data.loc[subject_data['need_extra_water_now']==1, 'prescribed_extra_supplement_amount']


    subject_data['training_status'] = 0
    subject_data.loc[~subject_data['scheduled_rig'].isnull(), 'training_status'] = 1
    subject_data.loc[~subject_data['session_location'].isnull(), 'training_status'] = 2

    subject_data['training_status_label'] = "Not scheduled"
    subject_data.loc[subject_data['training_status'] == 1, 'training_status_label'] = "Scheduled"
    subject_data.loc[subject_data['training_status'] == 2, 'training_status_label'] = "Training Started"

    #Calculated fields 
    subject_data['already_weighted'] = ~subject_data['weight'].isnull()
    subject_data['need_weight'] = ~subject_data['already_weighted'] | subject_data['need_reweight']

    subject_data['weight_status'] = "Already Weighted"
    subject_data.loc[subject_data['already_weighted'] == 0, 'weight_status'] = "Need Weight"
    subject_data.loc[subject_data['need_reweight'] == 1, 'weight_status'] = "REWEIGHT"


    return subject_data

### Datajoint configuration and Connection to DB

In [14]:
subject_data = get_subject_data()

In [15]:
pd.set_option('display.max_rows', 20)
subject_data

,subject_fullname,effective_date,subject_status,water_per_day,schedule,sr.subject_fullname,rfid,max_status.subject_fullname,last_status,s.subject_fullname,...,need_supplement,need_extra_water_now,water_status,need_water,current_need_water,training_status,training_status_label,already_weighted,need_weight,weight_status
0,efonseca_ef756_act123,2025-04-23,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,efonseca_ef756_act123,5C8B1724,efonseca_ef756_act123,2025-04-23,efonseca_ef756_act123,...,True,0,Need Supplement,1,1.0,1,Scheduled,False,True,Need Weight
1,efonseca_ef757_act124,2025-04-23,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,efonseca_ef757_act124,5C8B2047,efonseca_ef757_act124,2025-04-23,efonseca_ef757_act124,...,True,0,Need Supplement,1,1.0,1,Scheduled,False,True,Need Weight
2,efonseca_ef358_act125,2025-07-01,WaterRestrictionOnly,1.0,Water/Water/Water/Water/Water/Water/Water,None,None,efonseca_ef358_act125,2025-07-01,efonseca_ef358_act125,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
3,efonseca_ef037_act126,2025-07-01,WaterRestrictionOnly,1.0,Water/Water/Water/Water/Water/Water/Water,None,None,efonseca_ef037_act126,2025-07-01,efonseca_ef037_act126,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
4,jeremyjc_j084,2025-07-11,WaterRestrictionOnly,1.0,Nothing/Nothing/Nothing/Nothing/Nothing/Nothin...,None,None,jeremyjc_j084,2025-07-11,jeremyjc_j084,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,testuser_demo_add,2024-03-26,WaterRestrictionOnly,1.0,Train/Train/Train/Train/Train/Train/Train,None,None,testuser_demo_add,2024-03-26,testuser_demo_add,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
102,testuser_ironman_!,2024-03-01,WaterRestrictionOnly,1.0,Train/Train/Train/Train/Train/Train/Train,None,None,testuser_ironman_!,2024-03-01,testuser_ironman_!,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
103,testuser_monday_tria,2024-03-11,WaterRestrictionOnly,1.0,Train/Train/Train/Train/Train/Train/Train,None,None,testuser_monday_tria,2024-03-11,testuser_monday_tria,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight
104,testuser_sdfsdsaf,2025-03-19,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,None,None,testuser_sdfsdsaf,2025-03-19,testuser_sdfsdsaf,...,True,0,Need Supplement,1,1.0,0,Not scheduled,False,True,Need Weight


In [31]:
subject_data = subject_data.loc[~subject_data['subject_fullname'].str.contains('test'),:]
subjects_not_watered = subject_data.loc[subject_data['current_need_water'] > 0, ['subject_fullname', 'current_need_water']]
subjects_not_watered = subjects_not_watered.reset_index(drop=True)
subjects_not_watered['current_need_water'] = subjects_not_watered['current_need_water'].apply(lambda x: f"{x:.1f}")
subjects_not_watered = subjects_not_watered.head()
subjects_not_weighted = subject_data.loc[subject_data['need_weight'], ['subject_fullname', 'need_weight']]
subjects_not_weighted = subjects_not_weighted.reset_index(drop=True)
subjects_not_weighted = subjects_not_weighted.head()
subjects_not_trained = subject_data.loc[subject_data['training_status']==1, ['subject_fullname', 'scheduled_rig']]
subjects_not_trained = subjects_not_trained.reset_index(drop=True)
subjects_not_trained = subjects_not_trained.head()
subjects_not_trained

,subject_fullname,scheduled_rig
0,efonseca_ef756_act123,165A-miniVR-T-3
1,efonseca_ef757_act124,165A-miniVR-T-1
2,jeremyjc_j091,165I-Rig1-T
3,jeremyjc_j092,165I-Rig1-T
4,jeremyjc_j094,165I-Rig1-T


In [32]:
lab = dj.create_virtual_module('lab', 'u19_lab')
slack_configuration_dictionary = {
    'slack_notification_channel': ['alvaro_luna']
}
webhooks_list = []
query_slack_webhooks = [{'webhook_name' : x} for x in slack_configuration_dictionary['slack_notification_channel']]
webhooks_list += (lab.SlackWebhooks & query_slack_webhooks).fetch('webhook_url').tolist()

[2025-08-21 09:38:39,228][WARNING]: Reconnecting to MySQL server.


In [35]:
def slack_alert_message_format_weight_water(subjects_not_watered, subjects_not_weighted, subjects_not_trained):

    now = datetime.datetime.now()
    datestr = now.strftime('%d-%b-%Y %H:%M:%S')

    msep = dict()
    msep['type'] = "divider"

    #Title#
    m1 = dict()
    m1['type'] = 'section'
    m1_1 = dict()
    m1_1["type"] = "mrkdwn"
    m1_1["text"] = ':rotating_light: * Subjects Status Alert *'
    m1['text'] = m1_1

    #Info#
    m2 = dict()
    m2['type'] = 'section'
    m2_1 = dict()
    m2_1["type"] = "mrkdwn"

    m2_1["text"] = '*Subjects missing water:*' + '\n'
    for i in range(subjects_not_watered.shape[0]):
        m2_1["text"] += '*' + subjects_not_watered.loc[i, 'subject_fullname'] + '* : ' + str(subjects_not_watered.loc[i, 'current_need_water']) + ' ml\n'
    #m2_1["text"] += '\n'
    m2['text'] = m2_1

    m4 = dict()
    m4['type'] = 'section'
    m4_1 = dict()
    m4_1["type"] = "mrkdwn"

    m4_1["text"] = '*Subjects missing weighing:*' + '\n'
    for i in range(subjects_not_weighted.shape[0]):
        m4_1["text"] += '*' + subjects_not_weighted.loc[i, 'subject_fullname'] + '*\n' 
    m4['text'] = m4_1

    m5 = dict()
    m5['type'] = 'section'
    m5_1 = dict()
    m5_1["type"] = "mrkdwn"

    m5_1["text"] = '*Subjects missing training:*' + '\n'
    for i in range(subjects_not_trained.shape[0]):
        m5_1["text"] += '*' + subjects_not_trained.loc[i, 'subject_fullname'] + '* : ' + subjects_not_trained.loc[i, 'scheduled_rig'] + '\n'
    #
    m5['text'] = m5_1

    message = dict()
    message['blocks'] = [m1,msep,m2,msep,m4,msep,m5,msep]
    message['text'] = 'Subject Status Alert'

    return message

In [36]:
slack_json_message = slack_alert_message_format_weight_water(subjects_not_watered, subjects_not_weighted, subjects_not_trained)


#Send alert
for this_webhook in webhooks_list:
    su.send_slack_notification(this_webhook, slack_json_message)
    time.sleep(1)


In [37]:
current_directory = pathlib.Path(__file__).resolve()

NameError: name '__file__' is not defined